In [1]:
# Load in libraries

import warnings
warnings.filterwarnings('ignore')

#libraries for handling data
import pandas as pd
import numpy as np
import math
from sklearn import preprocessing
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import r2_score, mean_squared_error

#label encoders
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
from sklearn.preprocessing import OneHotEncoder
enc = OneHotEncoder()


#libraries for data visualization
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.pylab as pylab
%matplotlib inline
import seaborn as sns
sns.set_palette('Set2')

#libaries for modelling
# Regression Modelling Algorithms
import statsmodels.api as sm
#from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor , GradientBoostingRegressor

In [2]:
filepath = "/Users/reejungkim/Documents/Git/Production analysis/Online Retail.xlsx"
pd.ExcelFile(filepath).sheet_names

['Online Retail']

In [3]:
df = pd.read_excel(filepath, sheet_name = 'Online Retail')
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom


In [20]:
df.describe()

,Quantity,UnitPrice,TotalPurchase
count,541909.000000,541909.000000,541909.000000
mean,9.552250,4.611114,17.987795
std,218.081158,96.759853,378.810824
min,-80995.000000,-11062.060000,-168469.600000
25%,1.000000,1.250000,3.400000
50%,3.000000,2.080000,9.750000
75%,10.000000,4.130000,17.400000
max,80995.000000,38970.000000,168469.600000


In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 541909 entries, 0 to 541908
Data columns (total 9 columns):
 #   Column         Non-Null Count   Dtype         
---  ------         --------------   -----         
 0   InvoiceNo      541909 non-null  object        
 1   StockCode      541909 non-null  object        
 2   Description    540455 non-null  object        
 3   Quantity       541909 non-null  int64         
 4   InvoiceDate    541909 non-null  datetime64[ns]
 5   UnitPrice      541909 non-null  float64       
 6   CustomerID     541909 non-null  object        
 7   Country        541909 non-null  object        
 8   TotalPurchase  541909 non-null  float64       
dtypes: datetime64[ns](1), float64(2), int64(1), object(5)
memory usage: 37.2+ MB


In [18]:
df.CustomerID = df.CustomerID.astype(str)

In [14]:
df['TotalPurchase'] = df.Quantity * df.UnitPrice
df.head()

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPurchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
2,536365,84406B,CREAM CUPID HEARTS COAT HANGER,8,2010-12-01 08:26:00,2.75,17850.0,United Kingdom,22.00
3,536365,84029G,KNITTED UNION FLAG HOT WATER BOTTLE,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34
4,536365,84029E,RED WOOLLY HOTTIE WHITE HEART.,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [15]:
df.groupby('Country')['TotalPurchase'].sum().sort_values(ascending=False)[:5]

Country
United Kingdom    8.187806e+06
Netherlands       2.846615e+05
EIRE              2.632768e+05
Germany           2.216982e+05
France            1.974039e+05
Name: TotalPurchase, dtype: float64

In [6]:
df.Country.unique()

array(['United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany',
       'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal',
       'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland',
       'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria',
       'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore',
       'Lebanon', 'United Arab Emirates', 'Saudi Arabia',
       'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA',
       'European Community', 'Malta', 'RSA'], dtype=object)

In [22]:
uk_data = df.loc[df.Country=='United Kingdom']

In [23]:
uk_data.head(2)

,InvoiceNo,StockCode,Description,Quantity,InvoiceDate,UnitPrice,CustomerID,Country,TotalPurchase
0,536365,85123A,WHITE HANGING HEART T-LIGHT HOLDER,6,2010-12-01 08:26:00,2.55,17850.0,United Kingdom,15.30
1,536365,71053,WHITE METAL LANTERN,6,2010-12-01 08:26:00,3.39,17850.0,United Kingdom,20.34


In [24]:
uk_data_group=uk_data.groupby('CustomerID').agg({'InvoiceDate': lambda date: (date.max() - date.min()).days,
                                        'InvoiceNo': lambda num: len(num),
                                        'Quantity': lambda quant: quant.sum(),
                                        'TotalPurchase': lambda price: price.sum()})

uk_data_group

,InvoiceDate,InvoiceNo,Quantity,TotalPurchase
CustomerID,,,,
12346.0,0,2,0,0.00
12747.0,366,103,1275,4196.01
12748.0,372,4642,24210,29072.10
12749.0,209,231,1422,3868.20
12820.0,323,59,722,942.34
...,...,...,...,...
18281.0,0,7,54,80.82
18282.0,118,13,98,176.60
18283.0,333,756,1397,2094.88


In [25]:
uk_data_group.columns=['num_days','num_transactions','num_units','spent_money']
uk_data_group.head()

,num_days,num_transactions,num_units,spent_money
CustomerID,,,,
12346.0,0,2,0,0.00
12747.0,366,103,1275,4196.01
12748.0,372,4642,24210,29072.10
12749.0,209,231,1422,3868.20
12820.0,323,59,722,942.34


#CLTV = ((Average Order Value x Purchase Frequency)/Churn Rate) x Profit margin.
#Customer Value = Average Order Value * Purchase Frequency

In [26]:
uk_data_group['avg_order_value']=uk_data_group['spent_money']/uk_data_group['num_transactions']

In [27]:
uk_data_group.head()

,num_days,num_transactions,num_units,spent_money,avg_order_value
CustomerID,,,,,
12346.0,0,2,0,0.00,0.000000
12747.0,366,103,1275,4196.01,40.737961
12748.0,372,4642,24210,29072.10,6.262839
12749.0,209,231,1422,3868.20,16.745455
12820.0,323,59,722,942.34,15.971864


In [30]:
uk_data_group.shape

(3951, 5)

In [31]:
purchase_frequency=sum(uk_data_group['num_transactions'])/uk_data_group.shape[0]
purchase_frequency

125.40572007086813

In [32]:
repeat_rate= (
        uk_data_group[uk_data_group.num_transactions > 1].shape[0]/uk_data_group.shape[0]
)

In [34]:
#Churn Rate
churn_rate=1-repeat_rate

In [35]:
purchase_frequency,repeat_rate,churn_rate

(125.40572007086813, 0.9807643634522906, 0.019235636547709434)

In [36]:
# Approximated Profit Margin
#assume the business has approx 5% profit on the total sale.

uk_data_group['profit_margin']=uk_data_group['spent_money']*0.05

In [39]:
# Customer Value
uk_data_group['CLV']=(uk_data_group['avg_order_value']*purchase_frequency)/churn_rate


In [78]:
#Customer Lifetime Value
uk_data_group['cust_lifetime_value']=uk_data_group['CLV']*uk_data_group['profit_margin']


In [79]:
uk_data['month_yr'] = uk_data['InvoiceDate'].apply(lambda x: x.strftime('%Y-%m'))

In [91]:
uk_data.sort_values(by='InvoiceDate', ascending=True, inplace=True)

In [92]:
sale = uk_data.pivot_table(index=['CustomerID'],columns=['month_yr'],
                           values='TotalPurchase',aggfunc='sum',fill_value=0).reset_index()

In [99]:
sale.head()

month_yr,CustomerID,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12
0,12346.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,12747.0,706.27,303.04,0.00,310.78,0.00,771.31,376.30,0.00,301.70,0.00,675.38,312.73,438.50
2,12748.0,4177.68,418.77,389.64,1011.94,1100.37,2224.42,2006.26,1113.27,659.42,4218.65,1385.84,9295.57,1070.27
3,12749.0,0.00,0.00,0.00,0.00,0.00,782.10,0.00,0.00,1750.45,0.00,0.00,572.59,763.06
4,12820.0,0.00,170.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,217.77,343.76,0.00,210.35


In [70]:
#summation of customer value of each row across all columns except first two columns(month_yr and CustomerID)
sale['CLV']=sale.iloc[:,2:].sum(axis=1)

In [71]:
sale.head()

month_yr,CustomerID,2010-12,2011-01,2011-02,2011-03,2011-04,2011-05,2011-06,2011-07,2011-08,2011-09,2011-10,2011-11,2011-12,CLV
0,12346.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00
1,12747.0,706.27,303.04,0.00,310.78,0.00,771.31,376.30,0.00,301.70,0.00,675.38,312.73,438.50,6979.48
2,12748.0,4177.68,418.77,389.64,1011.94,1100.37,2224.42,2006.26,1113.27,659.42,4218.65,1385.84,9295.57,1070.27,49788.84
3,12749.0,0.00,0.00,0.00,0.00,0.00,782.10,0.00,0.00,1750.45,0.00,0.00,572.59,763.06,7736.40
4,12820.0,0.00,170.46,0.00,0.00,0.00,0.00,0.00,0.00,0.00,217.77,343.76,0.00,210.35,1884.68


In [27]:
X=sale[['Dec-2011','Nov-2011', 'Oct-2011','Sep-2011','Aug-2011','Jul-2011']]
y=sale[['CLV']]

In [32]:
#split training set and test set
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y,random_state=0)

In [33]:
# import model
from sklearn.linear_model import LinearRegression

# instantiate
linreg = LinearRegression()

# fit the model to the training data (learn the coefficients)
linreg.fit(X_train, y_train)

# make predictions on the testing set
y_pred = linreg.predict(X_test)